<img src="https://github.com/CropEvol/lecture/blob/master/ILAS_2022/01/images/banner.png?raw=true" alt="ILASセミナー「ゲノム博物学入門」" height="100px" align="middle">

# RNA-seq解析：PythonでBLAST検索

<div align="right"><a href="https://github.com/CropEvol/lecture#section1">実習表ページに戻る</a></div>

## はじめに

　前回、Pythonの基礎（変数、関数）とデータの読み込み、条件に合うデータの取り出し、グラフ描画を勉強しました。

　今回、以前勉強したBLAST検索をPythonで実行する方法と、そのBLAST検索を複数個の遺伝子すべてに対して実行する方法（処理の自動化方法）を勉強していきます。


## 相同性検索（BLAST検索）

　手元にある塩基配列とよく似た配列（相同配列）をデータベースから探す方法です。以前のセミナーで、[NCBIのBLAST検索](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastx&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome)をおこなってもらいました。

<img src="https://github.com/CropEvol/lecture/blob/master/ILAS_2022/RNAseq_intro/blast_images.png?raw=true" height="250px" align="middle">

## 今回の実習内容の流れ
1. PythonでBLAST検索する
1. 複数個の遺伝子をBLAST検索する
  1. Python基礎: 繰り返し構文
  1. BLAST検索の繰り返し処理
1. 前回の遺伝子発現解析データセットの遺伝子をBLAST検索してみよう
  1. 遺伝子名と遺伝子配列を取り出す
  1. 遺伝子名と遺伝子配列を同時に繰り返し処理する
  1. BLAST検索する
  1. `FDR<0.05`の遺伝子のみをBLAST検索する

## 1. PythonでBLAST検索する

　PythonでBLAST検索をおこなうには、生物学データ処理ライブラリ**Biopython**を使用します。

　前回使用した表計算ライブラリpandas　やグラフ描画ライブラリmatplotlib　とは異なり、BiopythonはGoogle Colaboratory（以下、Colab）にはインストールされていません。ColabでBiopythonを使用するには、まずインストールする必要があります。

　そのほかに、Colab上でBLAST検索をするために、以下のことをしています。
- NCBI Swissprotデータベースをダウンロード
- BLASTソフトウェアをダウンロード

<small>※ BLAST検索時に、NCBI上のデータベースにアクセスすることも可能ですが、検索に時間がかかるため、Colabにデータベースをダウンロードし、Colab上でBLAST検索できるようにします。</small>

In [ ]:
!pip install -q biopython
!wget -q -O - "https://ftp.ncbi.nlm.nih.gov/blast/db/swissprot.tar.gz" | tar zxf -
!wget -q -O - "https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.13.0+-x64-linux.tar.gz" | tar zxf - 
!cp -f /content/ncbi-blast-2.13.0+/bin/* /usr/bin

　BLAST検索プログラムをイチから作るのは大変なので`run_blast`という関数を準備しています。

<small>※ この関数に先ほどインストールしたBiopythonを使用しています。Biopythonはこれ以降表舞台に出てきませんが、`run_blast`関数を呼び出したときに裏方として毎回活躍しています。</small>

In [ ]:
import io
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIXML
from Bio.Blast import Applications

def run_blast(seq, db="swissprot"):
  # 関数に与えられたDNA配列のフォーマットを変換
  with open("seq.fa", "w") as out:
    SeqIO.write(SeqRecord(Seq(seq), id="sequence"), out, "fasta")
  # BLASTxの実行
  handle = Applications.NcbiblastxCommandline(query="seq.fa", db=db, outfmt=5)()[0]
  recs = NCBIXML.parse(io.StringIO(handle))
  for rec in recs:
    # 検索結果がひとつ以上のとき、ベストスコアの結果のみ取得
    for alignment in rec.alignments:
      for hsp in alignment.hsps:
        return alignment.accession, alignment.hit_def
    # 検索結果がNot Foundのとき
    return None, None

print('使い方: run_blast(<遺伝子配列>)')
print('戻り値: <BLAST_ID>, <BLAST_Full (詳細)>')
print('コード例: blast_id, blast_full = run_blast("ATGCCTATAG")')

　BLAST検索の準備ができたので、一個の遺伝子のBLAST検索を試してみましょう。

```python
# BLAST検索: run_blast()の使い方
<BLAST検索結果のID>, <BLAST検索結果の詳細> = run_blast(<遺伝子配列>)
```

In [ ]:
# 使用するDNA配列
gene = "GGGTGCGTGCGTGCGCCGGAGAGCCGGGGCGGCCGCCGGGGTTGTGTCGTCCTCCATGCTCACGATGTGTACTTACAGCGTTCAAAACTCGCGCGATCACGGCGCGGCCAGGCGCCACGCCGAGGTGGCGATCAGCGGGCGCGTGTGCCATCCCAGCGTCAGGCAGCCTTCCTTCTCCTCGACCTTGTACCCGTCGCCGCCGGCGAAGAGCGCCAGCAGCGTGCTCGCCTGCTTGTAGGCATTGGACCCCAGGTGGACGGTCTCGAACCCGGCGTTGCCCAGGCGGTTCCGCCACTGCCCCAGGGTCTCGTGGCGCTCTGTGCGCTCCGCTCCCTCGCAGGCCACCACGTTGCAGATCTGCCGGCCGAGGTACACCTCGGACATGACCTGGTCCGTGCCGGCCGCGGCAGGAGCGGCAGCCGGCCCCGATGAGACTTCGGATGGGCCGCCGCCGGAGCTGCCGCCCTCCAGAGAATCGAACATGGTGGAGTAGTAGTGCAGGGACTCGGTGAAGCGGTCCAGGAATGTGCCGGAGTTGTGGTTGGCCTCCTGCTCCACCACGGTGACGATCCTCGGCCGCACGGCGCGCACGGTGCCCAGGACCTTCTCCAGGGCGCCGGGCTGCGCCAGCAGCCGGTGCATCTCGAAGACTGAGTTGACGGCGATTACCTCCGGCTCCTCGTTCGGGTCCTCCTCGCCCTCCGGCTGCAGCATGAACGGCTCCAGGTCCGCGAGCGTGGCGGCGACGAGGCCGCGGTACTGGAAGTCGACGCGGATGGTGTGCGCGAACTGGGCGAGCTTCCAGCCCACCTGCTGCAGGGCGTCGGTCTCGTCCGGCTGCGGGGGTCCGACGCCGGTGAGGCGGAACGAGGGAGGGCCGCCGGGACGGAGGGCCAGTGCCTGGAGGAGGGCGGGCCACTGCATCCCCTGCTTGATGCCGAAGTCGACGACGTGCACGCGGCGGCAGCCGGCGAAGGCCTCCAGGATGGCCTGGTTGGCGGTGAAGTGGGCGAACTTGAGGTAGGGGCAGGACTCGTAGAAGTGCGCGTGGAGGAGGTCGGC"

# BLAST検索の実行
blast_id, blast_full = run_blast(gene)

# BLAST検索結果
print(blast_id)
print(blast_full)

## 2. 複数個の遺伝子をBLAST検索する

　ここから複数個の遺伝子をどのようにして処理（BLAST検索）するか勉強していきます。

　下のコードセルに5つの遺伝子配列を用意しました。それら遺伝子配列は変数`genes`にリスト型データとして入っています。

In [ ]:
genes = ["CATATCAATATGCATGGGTAATCCCTCTTCTCCCACTTCCAGTTATTATGTCAATGGGATTTGGCCTTATTCTTATTCCGACAGCAACAAAAAATCTTCGTCGCATATGGGCTTTTCCTAGTGTTTTACTCTTAAGTATAGCTATGGTATTCTCAGTTCAACTGTCTATTCAACAAATAAATGGAAGTTC",
          "AATCAATATACTTTTAATGTCGAATCGGGATTCACTAAGACAGAAATAAAGCATTGGGTCGAACTCTTCTTTGGTGTTAAGGTGGTAGCTGTGAATAGCCATCGACTACCTGGAAAAGGTAGAAGAATAGGACCTATTCTGGACCATACAATGCATTACAGACGTATGATCATTACCCTTCAACCGGGTTATTCTATTCCACTTCTAGATAGAGAAAAAAACTAA",
          "ATGACGGATTTAAACTTACCTTCTATTTTCGTGCCTTTAGTAGGCTTAGTATTTCCGGCAATTGCAATGACTTCTTTATTTCTTTATGTGCAAAAAAATAAGATTGTCTAG",
          "CCACCCGACGGACGAGGAGCTGGTGGCGGACTACCTCTGCGCGCGCGCGGCCGGGCGCGCGCCGCCGGTGCCCATCATCGCCGAGCTCGACCTCTACCGGTCCGACCCGTGGGAGCTCCCGGAGCGGGCGCTCTTCGGGGCGCGGGAGTGGTACTTCTTCACGCCGCGGGACCGCAAGTACCCCAACGGCTCCCGCCCCAGCCGGGCCGCCGGGGGCGGCTACTGGAAGGCCACCGGCGCCGACAGGCCCGTGGCGCACGCGGGCAGG",
          "ATGAAGAACTTCCTCGTCTTTGCCCTCCTCGCCGTTGTGGCGACAAGTACCATTGCACAGATGGAGACTAGCTGCATCCCTGGTTTGGAGAGACCATGGCAGCAGCAACCATTACCACCACAACAGTCATTTTCACAACAACCACCATTTCCACAACAACAACAACCATTACCTCAACAACCATCAATTTCGCAGCAACAACCACCATTTTTGCAGCAACAAGGACCACCATTTTCACAGCA",
]

　リスト内のそれぞれの遺伝子配列にアクセスするには、リスト内の通し番号（インデックス）を指定します。

<img src="https://github.com/CropEvol/lecture/blob/master/ILAS_2022/RNAseq_intro/list_data.png?raw=true" height="200px" align="middle">

In [ ]:
# 最初（インデックス0）の遺伝子
print(genes[0])
# インデックス1の遺伝子
print(genes[1])
# インデックス2の遺伝子
print(genes[2])
# インデックス3の遺伝子
print(genes[3])
# インデックス4の遺伝子
print(genes[4])


　遺伝子数が少ない場合はこのようなプログラムでも良いですが、遺伝子数が数百、数千、数万の場合、1遺伝子1行のコードを並べるのも大変です。

　そこで、もうすこし上手に各遺伝子にアクセスする方法を学びましょう。


### 2-1. Python基礎: 繰り返し構文
　**繰り返し構文（`for`構文）**というものを使うと、「リストデータから1遺伝子を取り出し、それを表示する」という処理を繰り返せます。

```python
for 変数 in リスト:
  print(変数)
```

In [ ]:
# 遺伝子をひとつずつ取り出す
for g in genes:
  # 取り出した遺伝子を表示
  print(g)

### 2-2. BLAST検索の繰り返し処理

　`for`構文中で、取り出した遺伝子配列を`run_blast`関数に与えてやれば、リスト内の遺伝子を順番にBLAST検索できます。

　下のコードセルの8行目にどのようなコードを書けばよいでしょうか？

```python
# ヒント: run_blast()の使い方
<BLAST検索結果のID>, <BLAST検索結果の詳細> = run_blast(<遺伝子配列>)
```


In [ ]:
# 遺伝子をひとつずつ取り出す
for g in genes:

  # 取り出した遺伝子を表示
  #print(g)

  # 取り出した遺伝子のBLAST検索実行と結果確認
  blast_id, blast_full = 
  print(blast_id, blast_full)

## 前回の遺伝子発現解析データセットの遺伝子をBLAST検索してみよう

　ここから前回のサンプルデータを扱います。

<img src="https://github.com/CropEvol/lecture/blob/master/ILAS_2022/RNAseq_intro/gene_expression_dataset.png?raw=true" align="middle" height="200">


In [ ]:
# サンプルデータのダウンロード
!wget -q -O "dataset.txt" https://raw.githubusercontent.com/CropEvol/lecture/master/data/ILAS_GEdataset.csv

# ライブラリ（追加機能）の準備
import pandas
# データの読み込み
df = pandas.read_csv("dataset.txt")
# 表示
df

各列の情報（今回扱う列のみ）
- `GeneID`: 遺伝子名
- `FDR`: 偽陽性確率
  - 2つのサンプル間の遺伝子発現量に差があるかどうかの指標として扱える
  - 遺伝子発現解析ではこの値が0.05より小さいときに発現量に差があると判定することが多い
- `GeneSeq`: 遺伝子の塩基配列

### 3-1. 遺伝子名と遺伝子配列を取り出す

　前回のおさらいとして、読み込んだデータフレーム`df`から`GeneID`列や`GeneSeq`列を取り出してみましょう。

 「`GeneSeq`列の取り出し」のほうはコードを追記してください。

In [ ]:
# GeneID列の取り出し
gene_id = df["GeneID"]
print(gene_id)

In [ ]:
# GeneSeq列の取り出し
gene_seq = 
print(gene_seq)

### 3-2. 遺伝子名と遺伝子配列を同時に繰り返し処理する

　2つの列（`gene_id`と`gene_seq`）を同時に繰り返し処理してみましょう。

```python
for 変数1, 変数2 in zip(リスト1, リスト2)
  print(変数1) # リスト1から取り出されたデータ
  print（変数2） # リスト2から取り出されたデータ
```


In [ ]:
# 二つのリストを同時に繰り返し処理する
for i, s in zip(gene_id, gene_seq):
  print(i)      # gene_idから取り出されたデータ
  print(s)     # gene_seqから取り出されたデータ

### 3-3. BLAST検索する
　for構文中で、`gene_seq`列から取り出された1個分の遺伝子配列を`run_blast`関数に与えれば、全遺伝子を順番にBLAST検索できます。

　なお、BLAST検索結果を表示する際に`gene_id`列から取り出した情報も表示すると、遺伝子名とBLAST検索結果を対応付けできます。

In [ ]:
# 繰り返し処理する
for i, s in zip(gene_id, gene_seq):
  #print(i)     # gene_idのデータ
  #print(s)    # gene_seqのデータ

  # BLAST検索
  blast_id, blast_full = run_blast(s)
  
  # 遺伝子名、BLAST検索結果のID、詳細を表示
  print(i, blast_id, blast_full)

### 3-4. `FDR<0.05`の遺伝子のみをBLAST検索する

　これまでに勉強したことを使って、`FDR<0.05`の遺伝子のみをBLAST検索してみてください。

手順:
1. 条件に合うデータのみが入った新規データフレームを作る
1. 新規データフレームの`gene_seq`列を取り出す
1. 繰り返し構文で、`gene_seq`列から1遺伝子ずつ取り出す
1. 取り出した1遺伝子を`run_blast`関数でBLAST検索する

```python
# ヒント: 条件に合うデータのみが入った新規データフレームを作る
新規データフレーム = データフレーム[条件式]
# 例: logFC<0を満たすデータのみを新規データフレームにする
df2 = df[ df["logFC"]<0 ]
```


## まとめ

　このテキストでは、次の内容を勉強しました。
- PythonでBLAST検索する方法
- `for`構文でデータ処理（BLAST検索）を自動化する方法

　ゲノム中に遺伝子は数万のオーダーで存在しています。ひとつ一つの遺伝子に対して、BLAST検索や遺伝子の長さを調べる、特定の塩基配列が含まれているかどうかを調べるといった作業をおこなうのは大変な仕事です。このような決められた処理を繰り返しおこなう場合、コンピュータにその処理を指示することで、解析をスピードアップさせることが可能です。

　コンピュータは与えられた指示を**高速**かつ**精確**に処理します。現代のようなビッグデータ時代では、コンピュータが多量に処理したデータを私たちが取り扱うというのが当たり前になってきています。そのようなデータのなかには、間違った指示により出力されたデータや与えた指示ではじゅうぶんに処理できなかったデータ（フィルタリングが不十分なデータ）が含まれていることもあります。私たちは、コンピュータが処理したデータを信じきるのではなく、信頼できるデータかどうかを判断する目を養う必要もあります。

<div align="right"><a href="https://github.com/CropEvol/lecture#section1">実習表ページに戻る</a></div>

---
#### 3-4のプログラム例

In [ ]:
# FDR<0.05を満たすデータを取り出し、新しいデータフレームにする
df_filtered = df[df["FDR"]<0.05]

# 遺伝子名と遺伝子配列の列を取り出す
gene_id = df_filtered["GeneID"]
gene_seq = df_filtered["GeneSeq"]

# 遺伝子名と遺伝子配列を順番に取り出す
for i, s in zip(gene_id, gene_seq):
  # BLAST検索
  blast_id, blast_full = run_blast(s)
  
  # 遺伝子名、BLAST検索結果のID、詳細を表示
  print(i, blast_id, blast_full)